In [7]:
import cv2
import numpy as np
import glob
from scipy.ndimage import measurements, interpolation

In [16]:
def find_principal_axis(image):
    """Find the centroid and principal axis of the particle using its contour."""
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        raise ValueError("No contours found in the image.")

    cnt = max(contours, key=cv2.contourArea)
    M = cv2.moments(cnt)
    centroid = (M["m10"] / M["m00"], M["m01"] / M["m00"])
    
    u20 = M["mu20"] / M["m00"]
    u02 = M["mu02"] / M["m00"]
    u11 = M["mu11"] / M["m00"]
    angle = 0.5 * np.arctan2(2 * u11, (u20 - u02))
    
    return centroid, angle

def rotate_image(image, angle, center):
    """Rotate the image around a center by a given angle."""
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR, borderValue=(255,255,255))
    return rotated

def resize_and_position(image, centroid, angle, target_points, image_size=(1000, 1000)):
    """Resize and position the image so that the principal axis endpoints match target_points."""
    y, x = np.nonzero(image)
    x_centered, y_centered = x - centroid[0], y - centroid[1]
    theta = -angle
    x_rotated = x_centered * np.cos(theta) - y_centered * np.sin(theta)
    min_x, max_x = x_rotated.min(), x_rotated.max()
    scale_factor = (target_points[1][0] - target_points[0][0]) / (max_x - min_x)

    image_resized = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)

    new_centroid = (centroid[0] * scale_factor, centroid[1] * scale_factor)
    
    translation_x = target_points[0][0] - new_centroid[0] + (image_size[0] - image_resized.shape[1]) / 2
    translation_y = target_points[0][1] - new_centroid[1] + (image_size[1] - image_resized.shape[0]) / 2

    M = np.float32([[1, 0, translation_x], [0, 1, translation_y]])
    img_translated = cv2.warpAffine(image_resized, M, image_size)

    return img_translated

def reverse_if_necessary(image, centroid, angle):
    """Reverse the image if the length under the principal axis is longer."""
    rows, cols = image.shape
    x_centroid, y_centroid = int(centroid[0]), int(centroid[1])

    mask = np.zeros_like(image, dtype=np.uint8)
    cv2.line(mask, (0, y_centroid), (cols, y_centroid), 1, thickness=1)
    upper_mask = np.triu(mask, 1)
    lower_mask = np.tril(mask, -1)

    upper_area = np.sum(image[upper_mask == 1])
    lower_area = np.sum(image[lower_mask == 1])

    if lower_area > upper_area:
        image = np.flipud(image)

    return image

In [18]:
for filepath in glob.glob('*.png'):
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    centroid, angle = find_principal_axis(image)
    rotated = rotate_image(image, np.degrees(angle), centroid)
    resized_and_positioned = resize_and_position(rotated, centroid, angle, [(200, 500), (800, 500)])
    final_image = reverse_if_necessary(resized_and_positioned, centroid, angle)

    # Save or display the final_image
    cv2.imwrite('processed_' + filepath, final_image)